In [ ]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x_train = pd.read_csv(Config.train_path)
x_train.head()

In [ ]:
x_train['reservation_status_date']

In [ ]:
from HTML.preprocessing import preprocessing
x_train = preprocessing(x_train)

In [ ]:
x_train['revenue'] = x_train['adr'] * np.sum(x_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1) * x_train['is_canceled']

In [ ]:
len(x_train)

In [ ]:
x_train = add_arrival_date(x_train)

In [ ]:
# x_train['revenue'] = (x_train['children']+x_train['adults']+x_train['babies']) * np.sum(x_train[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1)

In [ ]:
# dummies = x_train[list(set(ratio+['arrival_date'])-set(['adr', 'is_canceled', 'reservation_status', 'reservation_status_date']))]
# dummies = x_train[['revenue', 'arrival_date']]
dummies = x_train[ratio+['revenue', 'arrival_date']]
#   dummies = pd.get_dummies(
#     data=x_train[nominal+ordinal+['arrival_date']],
#     columns=nominal+ordinal,
#     drop_first=True,
#     dummy_na=True)
#   dummies = dummies.fillna(dummies.mean())

In [ ]:
x_train_processed = dummies.groupby('arrival_date').sum()

In [ ]:
y_train = pd.read_csv(Config.train_label_path)
train = x_train_processed.merge(y_train, left_index=True, right_on='arrival_date')

In [ ]:
train.columns

In [ ]:
x = (train['adults'] + train['children'] + train['babies'])
plt.scatter(x, train['adr'], alpha=.5)
plt.show()
plt.scatter(x, train['label'], alpha=.5)
plt.show()

In [ ]:
plt.scatter((train['children']+train['adults']+train['babies'])*(train['stays_in_weekend_nights']+train['stays_in_week_nights']), train['label'])

In [ ]:
plt.scatter(train['adr'], train['label'])

In [ ]:
plt.plot((train['children']+train['adults']+train['babies']) / np.mean(train['children']+train['adults']+train['babies']), alpha=.5)
plt.plot(train['adr'] / np.mean(train['adr']), alpha=.5)

In [ ]:
plt.plot(train['adr'])

In [ ]:
plt.plot(train['revenue'])

In [ ]:
x_train.tail()

In [ ]:
train.tail()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2

In [ ]:
train.columns

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNetCV

In [ ]:
x = train[x_train_processed.columns.drop('adr')]
y = train['adr']
regr = ElasticNetCV(cv=5)
score = cross_val_score(regr, x, y.to_numpy(), cv=5, scoring='neg_mean_squared_error')
np.mean(score)

In [ ]:
np.mean(score)/np.mean(train['adr'])

In [ ]:
regr.fit()
regr.predict(train[x_train_processed.columns.drop('adr')])

In [ ]:
x = train[x_train_processed.columns]
y = train['label']

In [ ]:
np.sum(chi2(x, y)[1] < .01)

In [ ]:
np.argsort(chi2(x, y)[1])

In [ ]:
chi2(x, y)[1]

In [ ]:
ind = np.argsort(chi2(x, y)[1])[::-1][:np.sum(chi2(x, y)[1] < .01)]

In [ ]:
x.iloc[:,ind]

In [ ]:
regr = ElasticNetCV(cv=5)

In [ ]:
X = x.iloc[:,ind].to_numpy()
# X_tmp = x.iloc[:,ind].to_numpy()
# X = np.zeros((x.shape[0], 1))
# for i in range(1,2):
#     X = np.concatenate((X, X_tmp**i),axis=1)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
score = cross_val_score(regr, X, y.to_numpy(), cv=5, scoring='neg_mean_squared_error')

In [ ]:
np.mean(score)

In [ ]:
from HTML.save import save_prediction

In [ ]:
regr.fit(X, y.to_numpy())

In [ ]:
x_test = pd.read_csv(Config.test_path)
x_test.head()

In [ ]:
set(x_train.columns) - set(x_test.columns)

In [ ]:
# x_test['revenue'] = x_test['adr'] * np.sum(x_test[['stays_in_weekend_nights', 'stays_in_week_nights']], axis=1)

In [ ]:
x_train = add_arrival_date(x_train)

In [ ]:
dummies = x_train[ratio+['revenue', 'arrival_date']]
#   dummies = pd.get_dummies(
#     data=x_train[nominal+ordinal+['arrival_date']],
#     columns=nominal+ordinal,
#     drop_first=True,
#     dummy_na=True)
#   dummies = dummies.fillna(dummies.mean())

In [ ]:
x_train_processed = dummies.groupby('arrival_date').sum()

In [ ]:
y_train = pd.read_csv(Config.train_label_path)
train = x_train_processed.merge(y_train, left_index=True, right_on='arrival_date')

In [ ]:
label = np.array([min(max(0, np.round(l)), 9) for l in regr.predict(test)])

In [ ]:
save_prediction(arrival_date=)